In [1]:
import json
from pathlib import Path
import pandas as pd
import numpy as np
import cv2 # OpenCV for image operations
from typing import Any, Dict, List, Optional, Tuple

from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm # For progress bars in notebooks

# Make plots look nicer
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
base_path = Path('/Users/satwato.dey/Documents/assignment_data_bdd/')

# Define paths to the specific files and directories
image_path = base_path / 'bdd100k_images_100k/bdd100k/images/100k'
train_json_path = base_path / 'bdd100k_labels_release/bdd100k/labels/bdd100k_labels_images_train.json'
val_json_path = base_path / 'bdd100k_labels_release/bdd100k/labels/bdd100k_labels_images_val.json'

# Verify that the paths exist
assert base_path.exists(), f"Base path not found: {base_path}"
assert image_path.exists(), f"Image path not found: {image_path}"
assert train_json_path.exists(), f"Train JSON not found: {train_json_path}"
assert val_json_path.exists(), f"Validation JSON not found: {val_json_path}"

print("All paths are correctly configured.")

All paths are correctly configured.


In [6]:
# Load the training and validation JSON files
with open(train_json_path, 'r') as f:
    train_data_raw = json.load(f)

with open(val_json_path, 'r') as f:
    val_data_raw = json.load(f)

# Let's inspect the first entry of the training data
print(f"Total training samples: {len(train_data_raw)}")
print(f"Total validation samples: {len(val_data_raw)}")
print("\n--- Example of a single data entry ---")
# Using json.dumps for pretty printing
print(json.dumps(train_data_raw[0], indent=2))

Total training samples: 69863
Total validation samples: 10000

--- Example of a single data entry ---
{
  "name": "0000f77c-6257be58.jpg",
  "attributes": {
    "weather": "clear",
    "scene": "city street",
    "timeofday": "daytime"
  },
  "timestamp": 10000,
  "labels": [
    {
      "category": "traffic light",
      "attributes": {
        "occluded": false,
        "truncated": false,
        "trafficLightColor": "green"
      },
      "manualShape": true,
      "manualAttributes": true,
      "box2d": {
        "x1": 1125.902264,
        "y1": 133.184488,
        "x2": 1156.978645,
        "y2": 210.875445
      },
      "id": 0
    },
    {
      "category": "traffic light",
      "attributes": {
        "occluded": false,
        "truncated": false,
        "trafficLightColor": "green"
      },
      "manualShape": true,
      "manualAttributes": true,
      "box2d": {
        "x1": 1156.978645,
        "y1": 136.637417,
        "x2": 1191.50796,
        "y2": 210.875443
    

In [7]:
def flatten_keys(d, parent_key=''):
    keys = []
    for k, v in d.items():
        full_key = f"{parent_key}.{k}" if parent_key else k
        if isinstance(v, dict):
            keys.extend(flatten_keys(v, full_key))
        else:
            keys.append(full_key)
    return keys

In [8]:
object_classes = {}
for image in train_data_raw:
    for label in image['labels']:
        if "box2d" in label:
            if label["category"] in object_classes:
                object_classes[label["category"]] += 1
            else:
                object_classes[label["category"]] = 1
# Convert the

In [9]:
object_classes_internals = {}
for image in train_data_raw:
    for label in image['labels']:
        # if "box2d" in label:
            if label["category"] in object_classes_internals:
                object_classes_internals[label["category"]].update(flatten_keys(label))
            else:
                object_classes_internals[label["category"]] = set(flatten_keys(label))

In [10]:
object_classes_internals

{'traffic light': {'attributes.occluded',
  'attributes.trafficLightColor',
  'attributes.truncated',
  'box2d.x1',
  'box2d.x2',
  'box2d.y1',
  'box2d.y2',
  'category',
  'id',
  'manualAttributes',
  'manualShape'},
 'traffic sign': {'attributes.occluded',
  'attributes.trafficLightColor',
  'attributes.truncated',
  'box2d.x1',
  'box2d.x2',
  'box2d.y1',
  'box2d.y2',
  'category',
  'id',
  'manualAttributes',
  'manualShape'},
 'car': {'attributes.occluded',
  'attributes.trafficLightColor',
  'attributes.truncated',
  'box2d.x1',
  'box2d.x2',
  'box2d.y1',
  'box2d.y2',
  'category',
  'id',
  'manualAttributes',
  'manualShape'},
 'drivable area': {'attributes.areaType',
  'category',
  'id',
  'manualAttributes',
  'manualShape',
  'poly2d'},
 'lane': {'attributes.laneDirection',
  'attributes.laneStyle',
  'attributes.laneType',
  'category',
  'id',
  'manualAttributes',
  'manualShape',
  'poly2d'},
 'person': {'attributes.occluded',
  'attributes.trafficLightColor',
  '

In [11]:
object_classes_internals

{'traffic light': {'attributes.occluded',
  'attributes.trafficLightColor',
  'attributes.truncated',
  'box2d.x1',
  'box2d.x2',
  'box2d.y1',
  'box2d.y2',
  'category',
  'id',
  'manualAttributes',
  'manualShape'},
 'traffic sign': {'attributes.occluded',
  'attributes.trafficLightColor',
  'attributes.truncated',
  'box2d.x1',
  'box2d.x2',
  'box2d.y1',
  'box2d.y2',
  'category',
  'id',
  'manualAttributes',
  'manualShape'},
 'car': {'attributes.occluded',
  'attributes.trafficLightColor',
  'attributes.truncated',
  'box2d.x1',
  'box2d.x2',
  'box2d.y1',
  'box2d.y2',
  'category',
  'id',
  'manualAttributes',
  'manualShape'},
 'drivable area': {'attributes.areaType',
  'category',
  'id',
  'manualAttributes',
  'manualShape',
  'poly2d'},
 'lane': {'attributes.laneDirection',
  'attributes.laneStyle',
  'attributes.laneType',
  'category',
  'id',
  'manualAttributes',
  'manualShape',
  'poly2d'},
 'person': {'attributes.occluded',
  'attributes.trafficLightColor',
  '

In [12]:
DETECTION_CATEGORIES = object_classes.keys()

In [13]:
class GranularBddParser:
    """
    Parses BDD100k data, creating a rich feature set for deep analysis.
    """
    def __init__(self, json_path: Path):
        print(f"Loading data from {json_path}...")
        with open(json_path, 'r') as f:
            self.raw_data: List[Dict[str, Any]] = json.load(f)
        print(f"Loaded {len(self.raw_data)} image records.")

    def _get_drivable_polygons(self, labels: List[Dict]) -> Dict[str, List[Polygon]]:
        """Extracts drivable area polygons from a list of labels."""
        polygons = {'direct': [], 'alternative': []}
        for label in labels:
            if label.get('category') == 'drivable area':
                area_type = label.get('attributes', {}).get('areaType')
                if area_type in polygons and 'poly2d' in label:
                    try:
                        # Ensure polygon has at least 3 points to be valid
                        if len(label['poly2d'][0]['vertices']) >= 3:
                            poly = Polygon(label['poly2d'][0]['vertices'])
                            polygons[area_type].append(poly)
                    except Exception:
                        continue
        return polygons

    def _get_location_context(
        self, box: Dict, drivable_polys: Dict[str, List[Polygon]]
    ) -> str:
        """Determines if an object is on the main road, sidewalk, or off-road."""
        centroid = Point((box['x1'] + box['x2']) / 2, (box['y1'] + box['y2']) / 2)
        for poly in drivable_polys.get('direct', []):
            if poly.contains(centroid):
                return 'on_main_road'
        for poly in drivable_polys.get('alternative', []):
            if poly.contains(centroid):
                return 'on_sidewalk'
        return 'off_road'

    def _calculate_features(
        self, label: Dict, image_attrs: Dict
    ) -> Tuple[Optional[float], float, int]:
        """Calculates tightness_ratio, normalized_area, and difficulty_score."""
        box2d = label['box2d']
        box_area = (box2d['x2'] - box2d['x1']) * (box2d['y2'] - box2d['y1'])
        normalized_area = round(box_area / IMG_AREA, 6) if IMG_AREA > 0 else 0

        tightness_ratio = None
        if 'poly2d' in label:
            try:
                # Ensure polygon has at least 3 points to be valid
                if len(label['poly2d'][0]['vertices']) >= 3:
                    poly = Polygon(label['poly2d'][0]['vertices'])
                    if box_area > 0:
                        tightness_ratio = round(poly.area / box_area, 4)
            except Exception:
                tightness_ratio = None
        
        # Calculate difficulty score
        difficulty = 0
        attrs = label.get('attributes', {})
        if attrs.get('occluded'): difficulty += 1
        if attrs.get('truncated'): difficulty += 1
        if image_attrs.get('timeofday') == 'night': difficulty += 1
        if image_attrs.get('weather') in ['rainy', 'snowy', 'foggy']: difficulty += 1
        if tightness_ratio is not None and tightness_ratio < TIGHTNESS_THRESHOLD:
            difficulty += 1
            
        return tightness_ratio, normalized_area, difficulty

    def parse_to_dataframe(self) -> pd.DataFrame:
        """Parses the entire JSON file into a single, feature-rich DataFrame."""
        parsed_records = []
        for image_record in tqdm(self.raw_data, desc="Granular Parsing"):
            if 'labels' not in image_record:
                continue

            # Step 1: Extract contextual polygons for the whole image
            drivable_polys = self._get_drivable_polygons(image_record['labels'])
            
            # Step 2: Process each object label within this context
            for label in image_record['labels']:
                category = label.get('category')
                if category not in DETECTION_CATEGORIES or 'box2d' not in label:
                    continue

                location_context = self._get_location_context(label['box2d'], drivable_polys)
                
                tightness, norm_area, difficulty = self._calculate_features(
                    label, image_record.get('attributes', {})
                )

                record = {
                    'image_name': image_record['name'],
                    'category': category,
                    'x1': label['box2d']['x1'], 'y1': label['box2d']['y1'],
                    'x2': label['box2d']['x2'], 'y2': label['box2d']['y2'],
                    'weather': image_record['attributes']['weather'],
                    'scene': image_record['attributes']['scene'],
                    'timeofday': image_record['attributes']['timeofday'],
                    'occluded': label.get('attributes', {}).get('occluded', False),
                    'truncated': label.get('attributes', {}).get('truncated', False),
                    'traffic_light_color': label.get('attributes', {}).get('trafficLightColor', 'none'),
                    'location_context': location_context,
                    'tightness_ratio': tightness,
                    'normalized_area': norm_area,
                    'difficulty_score': difficulty,
                }
                parsed_records.append(record)

        return pd.DataFrame(parsed_records)

print("GranularBddParser class defined.")

GranularBddParser class defined.


In [ ]:
IMG_WIDTH, IMG_HEIGHT = 1280, 720
IMG_AREA = IMG_WIDTH * IMG_HEIGHT
TIGHTNESS_THRESHOLD = 0.6 

In [17]:
parser_train = GranularBddParser(train_json_path)
df_train_granular = parser_train.parse_to_dataframe()

# Display the first few rows and the info of the resulting DataFrame
print("\n--- Granular Training DataFrame ---")
display(df_train_granular.head())
print("\n")
df_train_granular.info()

Loading data from /Users/satwato.dey/Documents/assignment_data_bdd/bdd100k_labels_release/bdd100k/labels/bdd100k_labels_images_train.json...
Loaded 69863 image records.


Granular Parsing:   0%|          | 0/69863 [00:00<?, ?it/s]


--- Granular Training DataFrame ---


,image_name,category,x1,y1,x2,y2,weather,scene,timeofday,occluded,truncated,traffic_light_color,location_context,tightness_ratio,normalized_area,difficulty_score
0,0000f77c-6257be58.jpg,traffic light,1125.902264,133.184488,1156.978645,210.875445,clear,city street,daytime,False,False,green,off_road,None,0.002620,0
1,0000f77c-6257be58.jpg,traffic light,1156.978645,136.637417,1191.507960,210.875443,clear,city street,daytime,False,False,green,off_road,None,0.002781,0
2,0000f77c-6257be58.jpg,traffic sign,1101.731743,211.122087,1170.790370,233.566141,clear,city street,daytime,False,False,none,off_road,None,0.001682,0
3,0000f77c-6257be58.jpg,traffic sign,0.000000,0.246631,100.381647,122.825696,clear,city street,daytime,False,True,none,off_road,None,0.013351,1
4,0000f77c-6257be58.jpg,car,45.240919,254.530367,357.805838,487.906215,clear,city street,daytime,False,False,none,off_road,None,0.079151,0




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1286871 entries, 0 to 1286870
Data columns (total 16 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   image_name           1286871 non-null  object 
 1   category             1286871 non-null  object 
 2   x1                   1286871 non-null  float64
 3   y1                   1286871 non-null  float64
 4   x2                   1286871 non-null  float64
 5   y2                   1286871 non-null  float64
 6   weather              1286871 non-null  object 
 7   scene                1286871 non-null  object 
 8   timeofday            1286871 non-null  object 
 9   occluded             1286871 non-null  bool   
 10  truncated            1286871 non-null  bool   
 11  traffic_light_color  1286871 non-null  object 
 12  location_context     1286871 non-null  object 
 13  tightness_ratio      0 non-null        object 
 14  normalized_area      1286871 non-null  float64
 

In [20]:
df_train_granular['category'].unique()

array(['traffic light', 'traffic sign', 'car', 'person', 'bus', 'truck',
       'rider', 'bike', 'motor', 'train'], dtype=object)

In [18]:
df_train_granular[df_train_granular['image_name']=='1d055f0d-a6223073.jpg']

,image_name,category,x1,y1,x2,y2,weather,scene,timeofday,occluded,truncated,traffic_light_color,location_context,tightness_ratio,normalized_area,difficulty_score
207433,1d055f0d-a6223073.jpg,car,812.005972,328.324309,841.944358,354.270909,clear,city street,night,False,False,none,off_road,None,0.000843,1
207434,1d055f0d-a6223073.jpg,car,777.077852,332.316094,810.010077,362.254481,clear,city street,night,True,False,none,off_road,None,0.001070,2
207435,1d055f0d-a6223073.jpg,car,756.120981,333.314039,791.049099,359.260642,clear,city street,night,True,False,none,off_road,None,0.000983,2
207436,1d055f0d-a6223073.jpg,person,797.036778,384.209297,806.018292,385.207243,clear,city street,night,True,False,none,on_sidewalk,None,0.000010,2
207437,1d055f0d-a6223073.jpg,person,748.137411,331.318147,768.096338,381.215461,clear,city street,night,True,False,none,off_road,None,0.001081,2
207438,1d055f0d-a6223073.jpg,person,728.178487,323.334578,749.135359,374.229835,clear,city street,night,True,False,none,off_road,None,0.001157,2
207439,1d055f0d-a6223073.jpg,person,705.225724,321.338685,732.170273,381.215461,clear,city street,night,True,False,none,off_road,None,0.001751,2
207440,1d055f0d-a6223073.jpg,car,646.346896,327.326363,717.201078,385.207242,clear,city street,night,True,False,none,off_road,None,0.004450,2
207441,1d055f0d-a6223073.jpg,car,585.472178,331.318147,691.254478,396.184653,clear,city street,night,False,False,none,off_road,None,0.007445,1
207442,1d055f0d-a6223073.jpg,car,848.929980,335.309932,871.882744,353.272964,clear,city street,night,False,False,none,off_road,None,0.000447,1


In [15]:
parser_val = GranularBddParser(val_json_path)
df_val_granular = parser_val.parse_to_dataframe()

# Display the first few rows and the info of the resulting DataFrame
print("\n--- Granular valing DataFrame ---")
display(df_val_granular.head())
print("\n")
df_val_granular.info()

Loading data from /Users/satwato.dey/Documents/assignment_data_bdd/bdd100k_labels_release/bdd100k/labels/bdd100k_labels_images_val.json...
Loaded 10000 image records.


Granular Parsing:   0%|          | 0/10000 [00:00<?, ?it/s]


--- Granular valing DataFrame ---


,image_name,category,x1,y1,x2,y2,weather,scene,timeofday,occluded,truncated,traffic_light_color,location_context,tightness_ratio,normalized_area,difficulty_score
0,b1c66a42-6f7d68ca.jpg,traffic sign,1000.698742,281.992415,1040.626872,326.911560,overcast,city street,daytime,False,False,none,off_road,None,0.001946,0
1,b1c66a42-6f7d68ca.jpg,traffic sign,214.613695,172.190058,274.505889,229.586743,overcast,city street,daytime,False,False,none,off_road,None,0.003730,0
2,b1c66a42-6f7d68ca.jpg,traffic sign,797.314833,313.186265,829.756437,341.884608,overcast,city street,daytime,False,False,none,off_road,None,0.001010,0
3,b1c66a42-6f7d68ca.jpg,traffic sign,652.575363,303.204232,685.016968,315.681772,overcast,city street,daytime,False,False,none,off_road,None,0.000439,0
4,b1c66a42-6f7d68ca.jpg,traffic light,707.476543,311.938510,716.210821,328.159313,overcast,city street,daytime,False,False,green,off_road,None,0.000154,0




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185526 entries, 0 to 185525
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   image_name           185526 non-null  object 
 1   category             185526 non-null  object 
 2   x1                   185526 non-null  float64
 3   y1                   185526 non-null  float64
 4   x2                   185526 non-null  float64
 5   y2                   185526 non-null  float64
 6   weather              185526 non-null  object 
 7   scene                185526 non-null  object 
 8   timeofday            185526 non-null  object 
 9   occluded             185526 non-null  bool   
 10  truncated            185526 non-null  bool   
 11  traffic_light_color  185526 non-null  object 
 12  location_context     185526 non-null  object 
 13  tightness_ratio      0 non-null       object 
 14  normalized_area      185526 non-null  float64
 15  difficulty_scor

In [16]:
df_val_granular.to_parquet("../data/extracted_data_val.pq", index=False)

In [5]:
def parse_bdd_data(raw_data: list) -> pd.DataFrame:
    """
    Parses the raw BDD100k data format into a clean Pandas DataFrame.
    
    Args:
        raw_data: A list of dictionaries, where each dict represents an image and its labels.
        
    Returns:
        A Pandas DataFrame with one row per bounding box.
    """
    parsed_records = []
    # Use tqdm for a nice progress bar
    for image_record in tqdm(raw_data, desc="Parsing data"):
        image_name = image_record['name']
        attributes = image_record['attributes']
        
        # If there are no labels for an image, we can skip it or handle as needed
        if 'labels' not in image_record:
            continue
            
        for label in image_record['labels']:
            # We only care about object detection classes which have 'box2d'
            if 'box2d' not in label:
                continue
            
            box2d = label['box2d']
            record = {
                'image_name': image_name,
                'category': label['category'],
                'x1': box2d['x1'],
                'y1': box2d['y1'],
                'x2': box2d['x2'],
                'y2': box2d['y2'],
                'weather': attributes['weather'],
                'scene': attributes['scene'],
                'timeofday': attributes['timeofday']
            }
            parsed_records.append(record)
            
    return pd.DataFrame(parsed_records)

# Apply the parser to our training and validation data
df_train = parse_bdd_data(train_data_raw)
df_val = parse_bdd_data(val_data_raw)

print("Parsing complete!")

Parsing data:   0%|          | 0/69863 [00:00<?, ?it/s]

Parsing data:   0%|          | 0/10000 [00:00<?, ?it/s]

Parsing complete!


In [ ]:
DETECTION_CATEGORIES = list(object_classes.keys())

def parse_bdd_data_enhanced(raw_data: list) -> pd.DataFrame:
    """
    Parses the raw BDD100k data, focusing only on object detection labels
    and extracting valuable attributes for deeper analysis.
    
    Args:
        raw_data: A list of dictionaries from the BDD JSON file.
        
    Returns:
        A Pandas DataFrame with one row per relevant bounding box.
    """
    parsed_records = []
    for image_record in tqdm(raw_data, desc="Parsing enhanced data"):
        image_name = image_record['name']
        top_level_attributes = image_record['attributes']
        
        if 'labels' not in image_record:
            continue
            
        for label in image_record['labels']:
            category = label['category']
            
            # --- STRATEGY: FILTER AGGRESSIVELY ---
            # 1. Skip if it's not a detection category
            if category not in DETECTION_CATEGORIES:
                continue
            
            # --- STRATEGY: EXTRACT HIGH-VALUE ATTRIBUTES ---
            label_attributes = label.get('attributes', {}) # Use .get for safety

            record = {
                # Top-level info
                'image_name': image_name,
                'weather': top_level_attributes['weather'],
                'scene': top_level_attributes['scene'],
                'timeofday': top_level_attributes['timeofday'],
                
                # Essential label info
                'category': category,
                'x1': label['box2d']['x1'],
                'y1': label['box2d']['y1'],
                'x2': label['box2d']['x2'],
                'y2': label['box2d']['y2'],

                # High-value analytical attributes
                'occluded': label_attributes.get('occluded', 'N/A'),
                'truncated': label_attributes.get('truncated', 'N/A'),
                'traffic_light_color': label_attributes.get('trafficLightColor', 'none') # Default to 'none'
            }
            parsed_records.append(record)
            
    return pd.DataFrame(parsed_records)

# --- Re-run the parsing with our new function ---
df_train_enhanced = parse_bdd_data_enhanced(train_data_raw)
df_val_enhanced = parse_bdd_data_enhanced(val_data_raw)

print("Enhanced parsing complete!")
display(df_train_enhanced.head())

In [6]:
df_train

,image_name,category,x1,y1,x2,y2,weather,scene,timeofday
0,0000f77c-6257be58.jpg,traffic light,1125.902264,133.184488,1156.978645,210.875445,clear,city street,daytime
1,0000f77c-6257be58.jpg,traffic light,1156.978645,136.637417,1191.507960,210.875443,clear,city street,daytime
2,0000f77c-6257be58.jpg,traffic sign,1101.731743,211.122087,1170.790370,233.566141,clear,city street,daytime
3,0000f77c-6257be58.jpg,traffic sign,0.000000,0.246631,100.381647,122.825696,clear,city street,daytime
4,0000f77c-6257be58.jpg,car,45.240919,254.530367,357.805838,487.906215,clear,city street,daytime
...,...,...,...,...,...,...,...,...,...
1286866,fe189115-8dabb5b1.jpg,traffic sign,499.744761,290.551824,532.693937,305.528722,undefined,highway,daytime
1286867,fe189115-8dabb5b1.jpg,traffic sign,507.233210,230.644231,528.200867,253.109578,undefined,highway,daytime
1286868,fe189115-8dabb5b1.jpg,traffic sign,506.593579,261.346872,532.054306,288.305289,undefined,highway,daytime
1286869,fe189115-8dabb5b1.jpg,car,710.279394,315.263706,737.237811,336.231364,undefined,highway,daytime


In [7]:
# Display the first few rows and info of the training dataframe
print("--- Training DataFrame ---")
display(df_train.head())
print("\n")
df_train.info()

print("\n\n--- Validation DataFrame ---")
display(df_val.head())

--- Training DataFrame ---


,image_name,category,x1,y1,x2,y2,weather,scene,timeofday
0,0000f77c-6257be58.jpg,traffic light,1125.902264,133.184488,1156.978645,210.875445,clear,city street,daytime
1,0000f77c-6257be58.jpg,traffic light,1156.978645,136.637417,1191.507960,210.875443,clear,city street,daytime
2,0000f77c-6257be58.jpg,traffic sign,1101.731743,211.122087,1170.790370,233.566141,clear,city street,daytime
3,0000f77c-6257be58.jpg,traffic sign,0.000000,0.246631,100.381647,122.825696,clear,city street,daytime
4,0000f77c-6257be58.jpg,car,45.240919,254.530367,357.805838,487.906215,clear,city street,daytime




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1286871 entries, 0 to 1286870
Data columns (total 9 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   image_name  1286871 non-null  object 
 1   category    1286871 non-null  object 
 2   x1          1286871 non-null  float64
 3   y1          1286871 non-null  float64
 4   x2          1286871 non-null  float64
 5   y2          1286871 non-null  float64
 6   weather     1286871 non-null  object 
 7   scene       1286871 non-null  object 
 8   timeofday   1286871 non-null  object 
dtypes: float64(4), object(5)
memory usage: 88.4+ MB


--- Validation DataFrame ---


,image_name,category,x1,y1,x2,y2,weather,scene,timeofday
0,b1c66a42-6f7d68ca.jpg,traffic sign,1000.698742,281.992415,1040.626872,326.911560,overcast,city street,daytime
1,b1c66a42-6f7d68ca.jpg,traffic sign,214.613695,172.190058,274.505889,229.586743,overcast,city street,daytime
2,b1c66a42-6f7d68ca.jpg,traffic sign,797.314833,313.186265,829.756437,341.884608,overcast,city street,daytime
3,b1c66a42-6f7d68ca.jpg,traffic sign,652.575363,303.204232,685.016968,315.681772,overcast,city street,daytime
4,b1c66a42-6f7d68ca.jpg,traffic light,707.476543,311.938510,716.210821,328.159313,overcast,city street,daytime
